# Парсинг и объединение данных о днях рождения с рабочим датасетом для задачи с Варфарином

In [ ]:
import os
import gdown
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# из библиотеки google.colab импортируем класс files
from google.colab import files

# создаем объект этого класса, применяем метод .upload()
uploaded = files.upload()

Saving ЦАТ_общий_(Казанцев) (1).xlsx to ЦАТ_общий_(Казанцев) (1).xlsx


In [ ]:
# Загрузка датасета Александра Казанцева
df = pd.read_excel("ЦАТ_общий_(Казанцев) (1).xlsx", sheet_name="СПРАВ_ФИО",skiprows=6)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1416 entries, 0 to 1415
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   №                    0 non-null      float64
 1   МО прикрепления      1416 non-null   object 
 2   Ф.И.О. пациента      1416 non-null   object 
 3   Пол                  1416 non-null   object 
 4   Дата рождения        1416 non-null   object 
 5   Изначанияе значения  1416 non-null   object 
 6   Без проблелов        1416 non-null   object 
 7   Приведённый шрифт    1416 non-null   object 
 8   id                   767 non-null    float64
 9   Unnamed: 9           1416 non-null   int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 110.8+ KB


нужны колонки ['Дата рождения ', 'Приведённый шрифт', 'id' ]

'id' представлен как float. Должен быть int

In [ ]:
dbackup=df.copy()  #сохранение резервной копии

co_lst = ['Дата рождения ', 'Приведённый шрифт', 'id' ]
# убираем ненужные колонки
df = df[co_lst]

# Новые названия колонок (для удобства)
df.columns = ['bd', 'fio', 'id']
# удаляем пустые id и приводим к int64
df = df.dropna()
df['id'] = df['id'].astype('int64')

# контроль
print(f'сначала {dbackup.shape}, не пустых {df.shape}')
df.head()


сначала (1416, 10), не пустых (767, 3)


,bd,fio,id
0,1976-12-07 00:00:00,Акбашева Альбина Гизыльхаковна,139797386435008
2,1960-01-11 00:00:00,Ананьев Владимир Алексеевич,139797386432992
3,1953-10-05 00:00:00,Аникина Валентина Никитична,139797384872704
4,1940-03-03 00:00:00,Балюра Елена Фатхутдиновна,140114979053744
5,1961-01-04 00:00:00,Биккулова Райля Исмагиловна,139821889782528


In [ ]:
# Загрузка рабочего датасета со всеми столбцами из текущей папки
data = pd.read_excel("dataset_wide.xlsx")
data.head()

,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d5,d6,d7,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,0,0,0,-10,13.602187,1,0,0.017956,-0.919637
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,0,0,0,-10,13.602187,1,0,0.017956,-0.919637
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,0,1,0,-17,5.502392,1,0,0.007255,-1.606331
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,0,0,0,-14,8.168148,1,0,0.010555,-1.312034
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,0,0,0,-14,8.168148,1,0,0.010555,-1.312034


In [ ]:
# проверка на присутствие ФИО из файла Александра в основных рабочих данных

# уникальные id в основных данных
dunq_ids = list(data['id'].unique())
# print(len(dunq_ids))

# Ищем список id из файла Александра, которые отсутствуют в основных
notinmain = []
for id_ in list(df['id'].unique()):
    if not id_ in dunq_ids:
        print(id_)
        notinmain.append(id_)

print(df[df['id'].isin(notinmain)])

140114978915648
139766009916416
139766009958960
                       bd                             fio               id
224   1949-12-16 00:00:00    Бондарева Надежда Алексеевна  140114978915648
524   1945-09-03 00:00:00  Зиннатуллин Марс Гиндуллинович  139766009916416
1179  1953-06-11 00:00:00     Фатыхова Гульсина Шакировна  139766009958960


In [ ]:
# Проверка типа колонки 'bd'
#--- df['bd'].apply(lambda x: tuple(x) if isinstance(x, str ) else x).unique()

# в колонку 'bdt' запишем тип значения в колонке 'bd'
df['bdt'] = df['bd'].apply(type)
print(df['bdt'].unique())
display(df[df['bdt']==str].head())


[<class 'datetime.datetime'> <class 'str'>]


,bd,fio,id,bdt
70,68 лет,Аглямова Разида Инсафовна,140361892059056,<class 'str'>
84,72 года,Акчурин Зинфар Зуфарович,140361892058928,<class 'str'>
88,69 лет,Александрова Валентина Михайловна,140361892556848,<class 'str'>
99,50 лет,Алпарова Гульнара Раиловна,140361892057264,<class 'str'>
103,68 лет,Алтынбаева Рамиля Талгатовна,140361892070000,<class 'str'>


In [ ]:
# количество и пример некорректных дат рождения
bd_as_age = list(df[df['bdt']==str]['id'])
print(f'неверных дат рождения: {len(bd_as_age)}')
print(bd_as_age[-10:])

неверных дат рождения: 61
[140361892069568, 140361892550448, 140361892550704, 140361892060336, 140361892551984, 140361892071440, 140361892070576, 140361892552368, 140361892549040, 140361892548656]


По таким записям нельзя будет вычислить знак зодиака.

Дату рождения для таких записей можно заполнить
- либо какой-то условной датой,
- либо вычислить дату рождения приблизительно.

Для получения (приблизительных) дат рождения по возрасту предлагается такое решение: из даты первого анализа вычесть возраст в годах

In [ ]:
# Получение приблизительных дат рождения по первой дате анализа

# выделяем записи без правильных дат рождения из основных данных
da_ = data.loc[data['id'].isin(bd_as_age)][['id','age','dose','date_analyse']]

# убираем повторные записи
da_=da_.drop_duplicates(subset=['id','age','dose'])
print(da_.shape)
# da_.head()

# находим приблизительную дату рождения по дате анализа и возрасту
da_['relY'] = da_['age'].apply(lambda x: relativedelta(years=x))
da_['bdc'] = da_['date_analyse']-da_['relY']
da_.head()

(61, 4)


<ipython-input-19-0eef83ea1b63>:13: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  da_['bdc'] = da_['date_analyse']-da_['relY']


,id,age,dose,date_analyse,relY,bdc
0,140361892057136,57,7.500,2022-11-11,relativedelta(years=+57),1965-11-11
2,140361892057264,50,5.000,2022-11-22,relativedelta(years=+50),1972-11-22
3,140361892057392,53,6.250,2022-11-21,relativedelta(years=+53),1969-11-21
5,140361892057648,63,6.250,2022-10-27,relativedelta(years=+63),1959-10-27
6,140361892057904,74,4.375,2022-06-05,relativedelta(years=+74),1948-06-05


In [ ]:
# Объединение вычисленных дат рождения с данными ФИО по 'id'
# контроль
print('до:', df.shape, da_.shape)

df = pd.merge(df, da_, how='left', on=['id'])

# контроль
print('после:',df.shape, da_.shape)

display(df[df['bdt']==str])

до: (767, 4) (61, 6)
после: (767, 9) (61, 6)


,bd,fio,id,bdt,age,dose,date_analyse,relY,bdc
30,68 лет,Аглямова Разида Инсафовна,140361892059056,<class 'str'>,68.0,3.750,2022-05-13,relativedelta(years=+68),1954-05-13
39,72 года,Акчурин Зинфар Зуфарович,140361892058928,<class 'str'>,72.0,3.750,2022-09-29,relativedelta(years=+72),1950-09-29
41,69 лет,Александрова Валентина Михайловна,140361892556848,<class 'str'>,69.0,3.125,2022-03-25,relativedelta(years=+69),1953-03-25
47,50 лет,Алпарова Гульнара Раиловна,140361892057264,<class 'str'>,50.0,5.000,2022-11-22,relativedelta(years=+50),1972-11-22
49,68 лет,Алтынбаева Рамиля Талгатовна,140361892070000,<class 'str'>,68.0,3.750,2022-09-14,relativedelta(years=+68),1954-09-14
...,...,...,...,...,...,...,...,...,...
677,56 лет,Худайбердин Флюр Чынтимерович,140361892071440,<class 'str'>,56.0,6.250,2022-11-07,relativedelta(years=+56),1966-11-07
681,65 лет,Хусаинов Рамиль Биктимерович,140361892070576,<class 'str'>,65.0,2.500,2022-02-17,relativedelta(years=+65),1957-02-17
683,65 лет,Хусаинов Шамиль Миннулович,140361892552368,<class 'str'>,65.0,6.875,2022-11-06,relativedelta(years=+65),1957-11-06
732,53 года,Шаяхметов Тагир Касимович,140361892549040,<class 'str'>,53.0,5.000,2022-12-30,relativedelta(years=+53),1969-12-30


In [ ]:
# Сведение реальной и вычисленной даты рождения в одну колонку
df['birthdate'] = df['bd']   # заполнение реальными датами рождения
msk = df['bdt']==str         # фильтр для записей с некорректной датой рождения

# колонка для отобранных записей заполняется вычисленными датами рождения
df.loc[msk,'birthdate'] = df['bdc']
# контроль
df.loc[msk].head()

,bd,fio,id,bdt,age,dose,date_analyse,relY,bdc,birthdate
30,68 лет,Аглямова Разида Инсафовна,140361892059056,<class 'str'>,68.0,3.750,2022-05-13,relativedelta(years=+68),1954-05-13,1954-05-13 00:00:00
39,72 года,Акчурин Зинфар Зуфарович,140361892058928,<class 'str'>,72.0,3.750,2022-09-29,relativedelta(years=+72),1950-09-29,1950-09-29 00:00:00
41,69 лет,Александрова Валентина Михайловна,140361892556848,<class 'str'>,69.0,3.125,2022-03-25,relativedelta(years=+69),1953-03-25,1953-03-25 00:00:00
47,50 лет,Алпарова Гульнара Раиловна,140361892057264,<class 'str'>,50.0,5.000,2022-11-22,relativedelta(years=+50),1972-11-22,1972-11-22 00:00:00
49,68 лет,Алтынбаева Рамиля Талгатовна,140361892070000,<class 'str'>,68.0,3.750,2022-09-14,relativedelta(years=+68),1954-09-14,1954-09-14 00:00:00


In [ ]:
# контроль по произвольной записи
df[df['id']==140361892057264]

,bd,fio,id,bdt,age,dose,date_analyse,relY,bdc,birthdate
47,50 лет,Алпарова Гульнара Раиловна,140361892057264,<class 'str'>,50.0,5.0,2022-11-22,relativedelta(years=+50),1972-11-22,1972-11-22 00:00:00


In [ ]:
# Очистка датасета по ФИО от вспомогательных колонок
df.drop(columns=['bdt','age','dose','date_analyse','relY','bdc'], inplace=True)
# контроль
print(df.shape)
df.head()

(767, 4)


,bd,fio,id,birthdate
0,1976-12-07 00:00:00,Акбашева Альбина Гизыльхаковна,139797386435008,1976-12-07 00:00:00
1,1960-01-11 00:00:00,Ананьев Владимир Алексеевич,139797386432992,1960-01-11 00:00:00
2,1953-10-05 00:00:00,Аникина Валентина Никитична,139797384872704,1953-10-05 00:00:00
3,1940-03-03 00:00:00,Балюра Елена Фатхутдиновна,140114979053744,1940-03-03 00:00:00
4,1961-01-04 00:00:00,Биккулова Райля Исмагиловна,139821889782528,1961-01-04 00:00:00


In [ ]:
# рабочий бэкап
df.to_csv('df_birthday.csv', index=False)

### Объединение с основными данными

In [ ]:
# контроль размерностей до
print(f'до: {data.shape}, {df.shape}')

# data = pd.merge(data, df, how='left', on=['id'])
data = pd.merge(data, df[['id','birthdate']], how='left', on=['id'])

# контроль размерностей до
print(f'после: {data.shape}, {df.shape}')
display(data.head())

до: (1463, 43), (767, 4)
после: (1463, 44), (767, 4)


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d6,d7,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthdate
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11 00:00:00
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11 00:00:00
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,1,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22 00:00:00
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21 00:00:00
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21 00:00:00


In [ ]:
# Проверка корректности объединенных данных
# print(data.groupby(data.columns.tolist(),as_index=False).size()) # поиск дубликатов
print(data.notnull().sum())     # поиск пустых значений
# print(data2.groupby('id').birthday.count())

# пример нескольких записей с одним id
display(data[data['id']==140502024767792])

institution              1463
id                       1463
gender                   1463
age                      1463
dose                     1463
date_analyse             1463
prev_INR(MHO)            1463
atrial_fibrillation      1463
mitral_valve             1463
aortal_valve             1463
date_diff                1463
INR(MHO)                 1463
sample                   1463
pass_dose                1463
institution_id           1463
diagnos                  1463
sss                       422
date_from_1990           1463
ageix                    1463
agebin_0-9               1463
agebin_10-19             1463
agebin_20-29             1463
agebin_30-39             1463
agebin_40-49             1463
agebin_50-59             1463
agebin_60-69             1463
agebin_70-79             1463
agebin_80-89             1463
agebin_90+               1463
d1                       1463
d2                       1463
d3                       1463
d4                       1463
d5        

,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d6,d7,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthdate
729,ЕрмкЦРБ,140502024767792,0,67,2.5,2022-10-28,1.2,1,0,0,...,0,0,0,0,49.62406,2,0,0.049769,0.061354,1955-10-22 00:00:00
730,ЕрмкЦРБ,140502024767792,0,67,2.5,2022-12-01,2.1,1,0,0,...,0,0,0,0,49.62406,2,0,0.049769,0.061354,1955-10-22 00:00:00


In [ ]:
# Сохранение объединенного датасета
# data.to_excel(catalogs_path+'ЦАТ_общий_parsed_plus_minus.xlsx')

data.to_excel('dataset_wide_birthday.xlsx', index=False)

# Скопировать файл из рабочей ФС Colab в общую рабочую папку !!!


------------------------
Тесты
---

In [ ]:
dwb = pd.read_excel('dataset_wide_birthday.xlsx')
dwb.head()

,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d6,d7,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthday
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,1,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21
